# IBM Applied Data Science Capstone
## Part 4: SQL-Based Exploratory Data Analysis

**Objective:** Perform SQL queries on the dataset for structured analysis

**Author:** Son Nguyen

---


In [ ]:
import pandas as pd
import pandasql as psql
import warnings
warnings.filterwarnings('ignore')

# Load SpaceX dataset
df = pd.read_csv('../data/spacex_launches_cleaned.csv')
df['Date'] = pd.to_datetime(df['Date_UTC'])

# Add derived features
df['Landing_Success'] = (df['Core_Landing'] == 'Success').astype(int)
df['Launch_Success_Binary'] = df['Success'].fillna(0).astype(int)

print("=" * 60)
print("SQL-BASED EXPLORATORY DATA ANALYSIS")
print("=" * 60)
print(f"\n✓ Dataset loaded successfully!")
print(f"✓ Shape: {df.shape}")
print(f"✓ Total launches: {len(df)}")


SQL-BASED EXPLORATORY DATA ANALYSIS

✓ Dataset loaded successfully!
✓ Shape: (187, 25)
✓ Total launches: 187


## Query 1: Launch and Landing Success by Rocket Type

**Question:** What is the performance (launch success and landing success) of each rocket type?

**SQL Analysis:** This query groups launches by rocket type and calculates aggregate statistics.


In [2]:
q1 = """
SELECT 
    Rocket_Name,
    COUNT(*) as Total_Launches,
    SUM(Launch_Success_Binary) as Successful_Launches,
    ROUND(AVG(Launch_Success_Binary) * 100, 2) as Launch_Success_Rate_Pct,
    SUM(Landing_Success) as Successful_Landings,
    SUM(CASE WHEN Core_Landing != 'No Attempt' THEN 1 ELSE 0 END) as Landing_Attempts,
    ROUND(AVG(CASE WHEN Core_Landing = 'Success' THEN 1.0 ELSE 0.0 END) * 100, 2) as Landing_Success_Rate_Pct,
    AVG(Cost_Per_Launch) as Avg_Cost
FROM df
GROUP BY Rocket_Name
ORDER BY Total_Launches DESC
"""

result1 = psql.sqldf(q1, locals())
print("\n🚀 Rocket Performance Summary:")
print("=" * 80)
print(result1.to_string(index=False))
print("\nInsight: Falcon 9 dominates with highest launch count and success rates.")



🚀 Rocket Performance Summary:
Rocket_Name  Total_Launches  Successful_Launches  Launch_Success_Rate_Pct  Successful_Landings  Landing_Attempts  Landing_Success_Rate_Pct  Avg_Cost
    Unknown             187                  181                    96.79                  143               158                     76.47       0.0

Insight: Falcon 9 dominates with highest launch count and success rates.


## Query 2: Landing Success Trends Over Time

**Question:** How has first stage landing success rate evolved year by year?

**SQL Analysis:** This query calculates landing success rates by year to identify trends and improvements.


In [3]:
q2 = """
SELECT 
    Year,
    COUNT(*) as Total_Launches,
    SUM(CASE WHEN Core_Landing != 'No Attempt' THEN 1 ELSE 0 END) as Landing_Attempts,
    SUM(CASE WHEN Core_Landing = 'Success' THEN 1 ELSE 0 END) as Successful_Landings,
    ROUND(AVG(CASE WHEN Core_Landing = 'Success' THEN 1.0 ELSE 0.0 END) * 100, 2) as Landing_Success_Rate_Pct,
    SUM(Launch_Success_Binary) as Successful_Launches,
    ROUND(AVG(Launch_Success_Binary) * 100, 2) as Launch_Success_Rate_Pct
FROM df
GROUP BY Year
ORDER BY Year
"""

result2 = psql.sqldf(q2, locals())
print("\n📈 Landing Success Trends by Year:")
print("=" * 80)
print(result2.to_string(index=False))
print("\nInsight: Landing technology improved significantly after 2015, showing SpaceX's learning curve.")



📈 Landing Success Trends by Year:
 Year  Total_Launches  Landing_Attempts  Successful_Landings  Landing_Success_Rate_Pct  Successful_Launches  Launch_Success_Rate_Pct
 2006               1                 0                    0                      0.00                    0                     0.00
 2007               1                 0                    0                      0.00                    0                     0.00
 2008               2                 0                    0                      0.00                    1                    50.00
 2009               1                 0                    0                      0.00                    1                   100.00
 2010               2                 0                    0                      0.00                    2                   100.00
 2012               2                 0                    0                      0.00                    2                   100.00
 2013               3             

## Query 3: Core Reuse Impact on Landing Success

**Question:** Does reusing cores affect landing success rates?

**SQL Analysis:** Compare landing success rates between new and reused cores.


In [4]:
q3 = """
SELECT 
    CASE 
        WHEN Core_Reused = 1 THEN 'Reused Core'
        ELSE 'New Core'
    END as Core_Type,
    COUNT(*) as Total_Launches,
    SUM(CASE WHEN Core_Landing != 'No Attempt' THEN 1 ELSE 0 END) as Landing_Attempts,
    SUM(CASE WHEN Core_Landing = 'Success' THEN 1 ELSE 0 END) as Successful_Landings,
    ROUND(AVG(CASE WHEN Core_Landing = 'Success' THEN 1.0 ELSE 0.0 END) * 100, 2) as Landing_Success_Rate_Pct,
    ROUND(AVG(Launch_Success_Binary) * 100, 2) as Launch_Success_Rate_Pct
FROM df
WHERE Core_Landing != 'No Attempt'
GROUP BY Core_Type
ORDER BY Landing_Success_Rate_Pct DESC
"""

result3 = psql.sqldf(q3, locals())
print("\n♻️ Core Reuse Impact Analysis:")
print("=" * 80)
print(result3.to_string(index=False))
print("\nInsight: Reused cores show different landing success characteristics.")



♻️ Core Reuse Impact Analysis:
  Core_Type  Total_Launches  Landing_Attempts  Successful_Landings  Landing_Success_Rate_Pct  Launch_Success_Rate_Pct
Reused Core             107               107                  104                     97.20                    99.07
   New Core              51                51                   39                     76.47                    96.08

Insight: Reused cores show different landing success characteristics.


## Query 4: Geographic Performance Analysis

**Question:** Which launch regions have the best success rates?

**SQL Analysis:** Analyze launch and landing success rates by geographic region.


In [5]:
q4 = """
SELECT 
    Region,
    Location,
    COUNT(*) as Total_Launches,
    SUM(Launch_Success_Binary) as Successful_Launches,
    ROUND(AVG(Launch_Success_Binary) * 100, 2) as Launch_Success_Rate_Pct,
    SUM(CASE WHEN Core_Landing = 'Success' THEN 1 ELSE 0 END) as Successful_Landings,
    SUM(CASE WHEN Core_Landing != 'No Attempt' THEN 1 ELSE 0 END) as Landing_Attempts,
    ROUND(AVG(CASE WHEN Core_Landing = 'Success' THEN 1.0 ELSE 0.0 END) * 100, 2) as Landing_Success_Rate_Pct
FROM df
GROUP BY Region, Location
ORDER BY Total_Launches DESC
"""

result4 = psql.sqldf(q4, locals())
print("\n🌍 Geographic Performance Analysis:")
print("=" * 80)
print(result4.to_string(index=False))
print("\nInsight: Cape Canaveral (Florida) has the most launches and highest success rates.")



🌍 Geographic Performance Analysis:
          Region                    Location  Total_Launches  Successful_Launches  Launch_Success_Rate_Pct  Successful_Landings  Landing_Attempts  Landing_Success_Rate_Pct
         Florida              Cape Canaveral             154                  152                    98.70                  120               133                     77.92
      California Vandenberg Space Force Base              28                   27                    96.43                   23                25                     82.14
Marshall Islands               Omelek Island               5                    2                    40.00                    0                 0                      0.00

Insight: Cape Canaveral (Florida) has the most launches and highest success rates.


## Query 5: Payload Mass Impact on Landing Success

**Question:** Do heavier payloads affect landing success probability?

**SQL Analysis:** Analyze landing success rates across different payload mass categories.


In [6]:
q5 = """
SELECT 
    CASE 
        WHEN Payload_Mass_kg < 1000 THEN 'Light (<1k kg)'
        WHEN Payload_Mass_kg < 5000 THEN 'Medium (1-5k kg)'
        WHEN Payload_Mass_kg < 10000 THEN 'Heavy (5-10k kg)'
        ELSE 'Very Heavy (>10k kg)'
    END as Payload_Category,
    COUNT(*) as Total_Launches,
    AVG(Payload_Mass_kg) as Avg_Payload_Mass_kg,
    SUM(CASE WHEN Core_Landing = 'Success' THEN 1 ELSE 0 END) as Successful_Landings,
    SUM(CASE WHEN Core_Landing != 'No Attempt' THEN 1 ELSE 0 END) as Landing_Attempts,
    ROUND(AVG(CASE WHEN Core_Landing = 'Success' THEN 1.0 ELSE 0.0 END) * 100, 2) as Landing_Success_Rate_Pct
FROM df
WHERE Core_Landing != 'No Attempt'
GROUP BY Payload_Category
ORDER BY Avg_Payload_Mass_kg
"""

result5 = psql.sqldf(q5, locals())
print("\n📦 Payload Mass Impact Analysis:")
print("=" * 80)
print(result5.to_string(index=False))
print("\nInsight: Payload mass may affect fuel margins for landing, impacting success rates.")



📦 Payload Mass Impact Analysis:
    Payload_Category  Total_Launches  Avg_Payload_Mass_kg  Successful_Landings  Landing_Attempts  Landing_Success_Rate_Pct
      Light (<1k kg)              30           180.866667                   27                30                     90.00
    Medium (1-5k kg)              44          2867.447727                   37                44                     84.09
    Heavy (5-10k kg)              20          7256.600000                   18                20                     90.00
Very Heavy (>10k kg)              64         14259.656250                   61                64                     95.31

Insight: Payload mass may affect fuel margins for landing, impacting success rates.


## Query 6: Launch Site Performance Comparison

**Question:** Which specific launchpads have the best performance?

**SQL Analysis:** Detailed launchpad-level analysis for operational insights.


In [7]:
q6 = """
SELECT 
    Launchpad_Name,
    Region,
    COUNT(*) as Total_Launches,
    SUM(Launch_Success_Binary) as Successful_Launches,
    ROUND(AVG(Launch_Success_Binary) * 100, 2) as Launch_Success_Rate_Pct,
    SUM(CASE WHEN Core_Landing = 'Success' THEN 1 ELSE 0 END) as Successful_Landings,
    ROUND(AVG(CASE WHEN Core_Landing = 'Success' THEN 1.0 ELSE 0.0 END) * 100, 2) as Landing_Success_Rate_Pct,
    AVG(Payload_Mass_kg) as Avg_Payload_Mass_kg
FROM df
GROUP BY Launchpad_Name, Region
HAVING Total_Launches >= 5
ORDER BY Launch_Success_Rate_Pct DESC
"""

result6 = psql.sqldf(q6, locals())
print("\n🚀 Launchpad Performance Comparison:")
print("=" * 80)
print(result6.to_string(index=False))
print("\nInsight: LC-39A and LC-40 show excellent performance with high launch and landing success rates.")



🚀 Launchpad Performance Comparison:
 Launchpad_Name           Region  Total_Launches  Successful_Launches  Launch_Success_Rate_Pct  Successful_Landings  Landing_Success_Rate_Pct  Avg_Payload_Mass_kg
     KSC LC 39A          Florida              55                   55                   100.00                   48                     87.27          7402.745455
   CCSFS SLC 40          Florida              99                   97                    97.98                   72                     72.73          6785.934848
    VAFB SLC 4E       California              28                   27                    96.43                   23                     82.14          6976.178571
Kwajalein Atoll Marshall Islands               5                    2                    40.00                    0                      0.00            77.000000

Insight: LC-39A and LC-40 show excellent performance with high launch and landing success rates.


## Query 7: Year-over-Year Growth Analysis

**Question:** What is the year-over-year growth in launches and landing success rates?

**SQL Analysis:** Calculate growth metrics to show SpaceX's expansion and improvement.


In [8]:
q7 = """
WITH yearly_stats AS (
    SELECT 
        Year,
        COUNT(*) as Total_Launches,
        ROUND(AVG(Launch_Success_Binary) * 100, 2) as Launch_Success_Rate_Pct,
        ROUND(AVG(CASE WHEN Core_Landing = 'Success' THEN 1.0 ELSE 0.0 END) * 100, 2) as Landing_Success_Rate_Pct
    FROM df
    GROUP BY Year
),
yearly_growth AS (
    SELECT 
        a.Year,
        a.Total_Launches,
        b.Total_Launches as Previous_Year_Launches,
        (a.Total_Launches - b.Total_Launches) as Launch_Growth,
        a.Landing_Success_Rate_Pct,
        b.Landing_Success_Rate_Pct as Previous_Landing_Rate,
        (a.Landing_Success_Rate_Pct - b.Landing_Success_Rate_Pct) as Landing_Rate_Improvement
    FROM yearly_stats a
    LEFT JOIN yearly_stats b ON a.Year = b.Year + 1
)
SELECT 
    Year,
    Total_Launches,
    Previous_Year_Launches,
    Launch_Growth,
    Landing_Success_Rate_Pct,
    Previous_Landing_Rate,
    ROUND(Landing_Rate_Improvement, 2) as Landing_Rate_Improvement_Pct
FROM yearly_growth
ORDER BY Year
"""

result7 = psql.sqldf(q7, locals())
print("\n📊 Year-over-Year Growth Analysis:")
print("=" * 80)
print(result7.to_string(index=False))
print("\nInsight: Shows SpaceX's rapid expansion in launch frequency and continuous improvement in landing technology.")



📊 Year-over-Year Growth Analysis:
 Year  Total_Launches  Previous_Year_Launches  Launch_Growth  Landing_Success_Rate_Pct  Previous_Landing_Rate  Landing_Rate_Improvement_Pct
 2006               1                     NaN            NaN                      0.00                    NaN                           NaN
 2007               1                     1.0            0.0                      0.00                   0.00                          0.00
 2008               2                     1.0            1.0                      0.00                   0.00                          0.00
 2009               1                     2.0           -1.0                      0.00                   0.00                          0.00
 2010               2                     1.0            1.0                      0.00                   0.00                          0.00
 2012               2                     NaN            NaN                      0.00                    NaN                

## SQL EDA Summary

**Key Insights from SQL Analysis:**
1. **Rocket Performance**: Falcon 9 shows dominant performance with highest launch count
2. **Landing Evolution**: Significant improvement in landing success rates over time
3. **Core Reuse**: Reused cores show different performance characteristics
4. **Geographic**: Cape Canaveral has the highest launch frequency and success
5. **Payload Impact**: Payload mass affects landing success probability
6. **Growth Trends**: Continuous year-over-year improvements in both launch frequency and landing success

**These insights will guide our predictive modeling approach in the next notebook.**
